In [1]:
import folium
import json
import pandas as pd
from geopy.geocoders import Nominatim

In [2]:
geolocator = Nominatim()
location = geolocator.geocode("Place du XX Aout, Liege, Belgium")
hotelibis1 = geolocator.geocode("41 pl de la République Francaise, 4000 Liege, Belgium")
hotelibis2 = geolocator.geocode("Rue de l'Arbre Courte Joie 380, 4000 Liège, Belgium")
hotelramada = geolocator.geocode("Quai St Léonard 36, 4000 Liège, Belgium")

In [12]:
#geo_json_data = json.load(open('countries.geo.json'))
geo_json_data = r'countries.geo.json'
state_data = pd.read_csv(r'./brbrbrbrb.csv')

In [13]:
map_clq = folium.Map(location=[location.latitude, location.longitude])
map_clq.geo_json('countries.geo.json')

In [16]:
map_clq.geo_json(geo_path=geo_json_data, data=state_data,
             columns=['Country','Abstracts'],
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Number of abstracts',
             reset='True')
map_clq.create_map('test.html')

In [17]:
map_clq